In [1]:
import numpy as np
from simplifiedLikelihoods import Data,UpperLimitComputer
import pandas as pd

### Define MET bins and their order

In [2]:
metBins = [250,  280,  310,  340,  370,  400,  430,  470,  510, 550,  590,  640,  690,  
            740,  790,  840,  900,  960, 1020, 1090, 1160, 1250, 1400]

### Load CMS data

In [3]:
cmsYieldFile = '../AuxInfo/CMS-EXO-20-004-data/HEPData-ins1894408-v2-csv/Simplifiedlikelihood:Yields(Monojet).csv'
bgYields = np.genfromtxt(cmsYieldFile,delimiter=',',skip_header=5,
                          skip_footer=67,names=True,dtype=None,encoding=None)

dataYields = np.genfromtxt(cmsYieldFile,delimiter=',',skip_header=73,
                           names=True,dtype=None,encoding=None)

covarianceFile = '../AuxInfo/CMS-EXO-20-004-data/HEPData-ins1894408-v2-csv/Simplifiedlikelihood:covariancematrix(Monojet).csv'
cov = np.genfromtxt(covarianceFile,delimiter=',',skip_header=5,
                           names=True,dtype=None,encoding='UTF-8')

### Extract covariance matrix

In [4]:
srOrder = cov['Second_Bin'][:66]
covMatrix = np.zeros((len(srOrder),len(srOrder)))
for pt in cov:
    i = np.where(srOrder == pt['First_Bin'])
    j = np.where(srOrder == pt['Second_Bin'])
    covMatrix[i,j] = pt['Covariance']

### Get number of observed and expected (BG) events

In [5]:
nobs = [int(dataYields[dataYields['Bin'] == sr]['Data_yield'][0]) for sr in srOrder]
nbg = [float(bgYields[bgYields['Bin'] == sr]['Background_yield'][0]) for sr in srOrder]

### Load Recast Data

In [6]:
recastFile = '../scanResults.pcl'
recastData = pd.read_pickle(recastFile)

### Get all model points

In [7]:
models = []
mCols = ['Coupling','Mode','$m_{med}$','$m_{DM}$']
for row in recastData[mCols].values:
    m = dict(zip(mCols,row.tolist()))
    if m not in models:
        models.append(m)
# print(len(models))

### Loop over model points and compute UL on mu

In [8]:
ulComp = UpperLimitComputer()

for m in models:
    dfModel = recastData.loc[(recastData[list(m)] == pd.Series(m)).all(axis=1)]
    ns = []
    for sr in srOrder:
        dataset,ibin = sr.split('_')[1:]
        dataset = eval(dataset)
        ibin = eval(ibin.replace('bin',''))
        binLabel = 'bin_%1.1f_%1.1f' %(metBins[ibin],metBins[ibin+1])
        signalYield = dfModel[dfModel['Data-takingperiod'] == dataset][binLabel].iloc[0]
        ns.append(signalYield)
    ns = np.array(ns)    
    data = Data(observed=nobs, backgrounds=nbg, covariance=covMatrix, 
                nsignal=ns,deltas_rel=0.0)
    dataExp = Data(observed=[int(b) for b in nbg], backgrounds=nbg, covariance=covMatrix, 
                   nsignal=ns,deltas_rel=0.0)    
    try:
        ul = ulComp.getUpperLimitOnMu(data)
    except:
        print('Error computing ul for model:\n',m,'\n')
        ul = None
    try:
        ulExp = ulComp.getUpperLimitOnMu(dataExp)    
    except:
        print('Error computing ulExp for model:\n',m,'\n')
        ulExp = None
        
    recastData.loc[dfModel.index,'$\mu^{UL}_{obs}$'] = ul
    recastData.loc[dfModel.index,'$\mu^{UL}_{exp}$'] = ulExp

Error computing ul for model:
 {'Coupling': 'Axial', 'Mode': 'DM+QCDjets', '$m_{med}$': 100.0, '$m_{DM}$': 1.0} 



In [9]:
recastData

,Coupling,Mode,$m_{med}$,$m_{DM}$,Data-takingperiod,Luminosity (1/fb),bin_250.0_280.0,bin_250.0_280.0_ErrorPlus,bin_250.0_280.0_ErrorMinus,bin_280.0_310.0,...,Tauveto,Bjetveto,Photonveto,"$\Delta \phi (jet,p_{T}^{miss})>0.5$ rad",LeadingAK4jet$p_{T}>100$GeV,LeadingAK4jet$\eta<2.4$,HCALmitigation(jets),HCALmitigation($\phi^{miss}$),$\mu^{UL}_{obs}$,$\mu^{UL}_{exp}$
0,Axial,DM+QCDjets,100.0,1.0,2018,59.7,266750.640696,23342.595332,23342.595332,153465.890268,...,0.128637,0.114142,0.113911,0.113738,0.111625,0.110215,0.106031,0.087903,NaN,0.008518
1,Axial,DM+QCDjets,100.0,1.0,2017,41.5,226905.642311,21283.716214,21283.716214,143503.815319,...,0.119203,0.106148,0.105486,0.104797,0.103599,0.102431,0.102431,0.102431,NaN,0.008518
2,Axial,DM+QCDjets,100.0,1.0,2016,36.0,190685.055980,18107.717764,18107.717764,91546.150031,...,0.118993,0.104625,0.102602,0.102251,0.100548,0.099690,0.099690,0.099690,NaN,0.008518
3,Axial,DM+QCDjets,100.0,30.0,2018,59.7,180189.362690,15199.497261,15199.497261,109162.687659,...,0.117268,0.105628,0.104922,0.104608,0.103498,0.102965,0.099394,0.082533,0.017504,0.011222
4,Axial,DM+QCDjets,100.0,30.0,2017,41.5,146927.437719,13913.332402,13913.332402,84454.293714,...,0.119431,0.105503,0.104457,0.103698,0.103087,0.102372,0.102372,0.102372,0.017504,0.011222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,Axial,DM+QCDjets,2500.0,1000.0,2017,41.5,15.408337,1.247961,1.247961,11.234426,...,0.393093,0.345913,0.334076,0.333848,0.331586,0.330125,0.330125,0.330125,13.678508,7.140237
200,Axial,DM+QCDjets,2500.0,1000.0,2016,36.0,13.600328,1.202180,1.202180,8.887383,...,0.372587,0.331511,0.320217,0.319577,0.316115,0.316046,0.316046,0.316046,13.678508,7.140237
201,Axial,DM+QCDjets,2500.0,1500.0,2018,59.7,0.026031,0.004220,0.004220,0.020503,...,0.371544,0.321315,0.312096,0.311492,0.309542,0.309997,0.297973,0.258334,7084.055427,4820.342512
202,Axial,DM+QCDjets,2500.0,1500.0,2017,41.5,0.020605,0.003582,0.003582,0.018577,...,0.348293,0.299475,0.286645,0.284621,0.281382,0.279956,0.279956,0.279956,7084.055427,4820.342512


In [10]:
recastData.to_pickle(recastFile)